In [1]:
import apogee.tools.read as apread
from apogee.tools import bitmask
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import apogee.spec.window as wn
import os
import access_spectrum as acs
import reduce_dataset as rd
import polyfit as pf
import matplotlib
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

In [8]:
# Load in Red Clump sample
data = apread.rcsample()
ind = np.where(data['FE_H']!=-9999)
len(data['FE_H'][ind])

19936

In [5]:
pf.hist2d(data['FE_H'][ind],data['ALPHAFE'][ind],labels = ['Abundances in Red Clump Stars','$[Fe/H]$',r'$[\alpha/Fe]$','Number of Stars'],saveloc = '/Users/Natalie/Code/rcapogee-plots/tex/alpha_vs_fe.png')

(array([-0.82736659, -0.80210524, -0.77684389, -0.75158254, -0.72632119,
        -0.70105984, -0.67579849, -0.65053713, -0.62527578, -0.60001443,
        -0.57475308, -0.54949173, -0.52423038, -0.49896903, -0.47370768,
        -0.44844633, -0.42318498, -0.39792363, -0.37266227, -0.34740092,
        -0.32213957, -0.29687822, -0.27161687, -0.24635552, -0.22109417,
        -0.19583282, -0.17057147, -0.14531012, -0.12004877, -0.09478742,
        -0.06952606, -0.04426471, -0.01900336,  0.00625799,  0.03151934,
         0.05678069,  0.08204204,  0.10730339,  0.13256474,  0.15782609,
         0.18308744,  0.2083488 ,  0.23361015,  0.2588715 ,  0.28413285,
         0.3093942 ,  0.33465555,  0.3599169 ,  0.38517825,  0.4104396 ,
         0.43570095]),
 array([-0.16740933, -0.15690844, -0.14640755, -0.13590666, -0.12540578,
        -0.11490489, -0.104404  , -0.09390311, -0.08340222, -0.07290133,
        -0.06240044, -0.05189955, -0.04139866, -0.03089778, -0.02039689,
        -0.009896  ,  0.0006

In [ ]:
# Load in spectra
label = 'FE_H'
low = -0.5
up = -0.4
sliceinfo = [label,low,up]
sindx = rd.slice_data(data, sliceinfo)
sdata = data[sindx]
maskdata = acs.get_spectra_ap(sdata,ext = 3, header = False,indx = 1)
ints = [i for i in range(len(maskdata)) if not isinstance(maskdata[i],np.int16)]
maskdata = maskdata[ints]
maskdata = np.array(list(maskdata),dtype = np.int64)
specs = acs.get_spectra_asp(sdata,header = False)
specs = specs[ints]
errs = acs.get_spectra_asp(sdata,ext = 2, header = False)
errs = errs[ints]

In [ ]:
# Find fit residuals
O = 1
badcombpixmask= bitmask.badpixmask()+2**bitmask.apogee_pixmask_int("SIG_SKYLINE")
dir = './'
ressname = 'rcresiduals_order{0}_{1}_u{2}_d{3}.pkl'.format(O,label,low,up)
ngen = False
if os.path.isfile(dir+ressname) and ngen:
    ress = acs.pklread(dir+ressname)
elif not os.path.isfile(dir+ressname) or not ngen:
    ps = []
    ress = []
    discard = []
    for pix in range(len(specs[0])):
        mask,badspec = rd.pixmask_find(maskdata,pix,bitval = badcombpixmask)
        mdata = sdata[mask]
        mspecs = specs[mask]
        merrs = errs[mask]
        discard.append('{0} of {1}'.format(len(sdata)-len(mask[0]),len(sdata)))
        res = np.zeros(len(specs))
        try:
            p = pf.regfit((mdata['TEFF'],mdata['LOGG'],mdata['FE_H']),mspecs[:,pix],err = merrs[:,pix],order = O)
            ps.append(p)
            res[mask] = mspecs[:,pix] - pf.poly(p,(mdata['TEFF'],mdata['LOGG'],mdata['METALS']),order = O)
            ress.append(res)
        except LinAlgError:
            continue
    ress = np.array(ress)
    dfile = open('dis_'+ressname.split('.pkl')[0]+'.txt','wb')
    dfile.write("\n".join(discard))
    acs.pklwrite(ressname,ress) 

In [ ]:
# Plot residuals for each element
elems = ['Al','Ca','C','Fe','K','Mg','Mn','Na','Ni','N','O','Si','S','Ti','V']
for elem in elems:
    w = wn.read(elem,dr = 12,apStarWavegrid=False)
    th = wn.tophat(elem,dr=12,apStarWavegrid=False)
    th = np.where(th == True)
    nw = pf.normweights(w)[th]
    resids = []
    for star in range(len(ress[0])):
        resids.append(pf.genresidual(nw,ress[:,star][th]))
    acs.pklwrite('{0}_residuals_order{1}_{2}_u{3}_d{4}.pkl'.format(elem,O,label,low,up),resids)
    plt.figure(figsize = (10,8))
    plt.hist(resids,bins = 50,range=(-0.1,0.1))
    plt.xlim(-0.1,0.1)
    plt.xlabel('Weighted fit residual of star for {0}'.format(elem))
    plt.ylabel('Count')
    plt.title('Residual from TEFF, LOGG, METALS fit for {0}'.format(elem))
    plt.savefig(dir+'{0}_residuals_order{1}_{2}_u{3}_d{4}.png'.format(elem,O,label,low,up))

In [ ]:
# Plot results for Aluminum line
doplot = False
if doplot:
    pix = 6296
    res = ress[pix]
    mask,badspec = pixmask_find(maskdata,pix)
    mdata = sdata[mask]
    mspecs = specs[mask]
    merrs = errs[mask]
    Teff = np.arange(min(mdata['TEFF']),max(mdata['TEFF']))
    logg = np.arange(min(mdata['LOGG']),max(mdata['LOGG']),((max(mdata['LOGG']-min(mdata['LOGG'])))/len(Teff)))
    metals = np.arange(min(mdata['METALS']),max(mdata['METALS']),((max(mdata['METALS']-min(mdata['METALS'])))/len(Teff)))
    pl = np.where(merrs[:,pix] < 0.1)
    rpl = np.where(merrs[:,pix] > 0.1)
    plt.figure(figsize = (10,8))
    plt.subplot2grid((3,3),(0,0))
    plt.errorbar(mdata['TEFF'][pl],mspecs[:,pix][pl],yerr = merrs[:,pix][pl],fmt='.')
    plt.plot(Teff,poly_indep(p,(Teff,logg,metals),order = O),linewidth = 3)
    plt.plot(mdata['TEFF'][rpl],mspecs[:,pix][rpl],'.')
    plt.xlabel('T_EFF')
    plt.ylabel('Pixel {0}'.format(pix))
    plt.subplot2grid((3,3),(1,0))
    plt.plot(mdata['TEFF'][pl],res[pl],'.')
    plt.xlabel('T_EFF')
    plt.ylabel('Fit residuals')
    plt.subplot2grid((3,3),(2,0))
    plt.semilogx(errs[:,pix][pl],res[pl],'.')
    plt.xlabel('Uncertainty in Pixel {0}'.format(pix))
    plt.ylabel('Fit residuals')
    plt.subplot2grid((3,3),(0,1))
    plt.errorbar(mdata['LOGG'][pl],mspecs[:,pix][pl],yerr = merrs[:,pix][pl],fmt='.')
    plt.plot(logg,poly_indep(p,(Teff,logg,metals),order = O),linewidth = 3)
    plt.plot(mdata['LOGG'][rpl],mspecs[:,pix][rpl],'.')
    plt.xlabel('LOG(G)')
    plt.ylabel('Pixel {0}'.format(pix))
    plt.subplot2grid((3,3),(1,1))
    plt.plot(mdata['LOGG'][pl],res[pl],'.')
    plt.xlabel('LOG(G)')
    plt.ylabel('Fit residuals')
    plt.tight_layout()
    plt.subplot2grid((3,3),(0,2))
    plt.errorbar(mdata['METALS'][pl],mspecs[:,pix][pl],yerr = merrs[:,pix][pl],fmt='.')
    plt.plot(metals,poly_indep(p,(Teff,logg,metals),order = O),linewidth = 3)
    plt.plot(mdata['METALS'][rpl],mspecs[:,pix][rpl],'.')
    plt.xlabel('Metallicity')
    plt.ylabel('Pixel {0}'.format(pix))
    plt.subplot2grid((3,3),(1,2))
    plt.plot(mdata['METALS'][pl],res[pl],'.')
    plt.xlabel('Metallicity')
    plt.ylabel('Fit residuals')

In [ ]:
if doplot:
    fig = plt.figure(figsize = (10,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(mdata['TEFF'],mdata['LOGG'],specs[:,6296],'.')
    ax.plot(Teff,logg,poly_indep(p,(Teff,logg,metals)))

In [ ]:
from read_clusterdata import read_caldata
dat = read_caldata()

In [ ]:
dat['APOGEE_ID'] = dat['ID']

In [ ]:
#maskdata = acs.get_spectra_ap(dat,ext = 3, header = False,indx = 1)
#ints = [i for i in range(len(maskdata)) if not isinstance(maskdata[i],np.int16)]
#maskdata = maskdata[ints]
#maskdata = np.array(list(maskdata),dtype = np.int64)
specs,ind = acs.get_spectra_asp(dat,header = False)
#specs = specs[ints]
errs,ind = acs.get_spectra_asp(dat,ext = 2, header = False)
#errs = errs[ints]

In [ ]:
O = 1
dat = dat[ind]
Tkeep = np.where(dat['TEFF']!=0)
dat = dat[Tkeep]
specs = specs[Tkeep]
errs = errs[Tkeep]

In [ ]:
# Find fit residuals
dir = './'
ressname = 'cresiduals_order{0}.pkl'.format(O)
ngen = False
if os.path.isfile(dir+ressname) and ngen:
    ress = acs.pklread(dir+ressname)
elif not os.path.isfile(dir+ressname) or not ngen:
    ps = []
    ress = []
    discard = []
    for pix in range(len(specs[0])):
        #mask,badspec = rd.pixmask_find(maskdata,pix,bitval = badcombpixmask)
        #mdata = sdata[mask]
        #mspecs = specs[mask]
        #merrs = errs[mask]
        #discard.append('{0} of {1}'.format(len(sdata)-len(mask[0]),len(sdata)))
        res = np.zeros(len(specs))
        try:
            p = pf.regfit(dat['TEFF'],specs[:,pix],err = errs[:,pix],order = O)
            ps.append(p)
            res = specs[:,pix] - pf.poly(p,dat['TEFF'],order = O)
            ress.append(res)
            #plt.figure()
            #plt.plot(dat['TEFF'],specs[:,pix],'.')
            #plt.plot(dat['TEFF'],pf.poly(p,dat['TEFF'],order = O))
        except LinAlgError:
            continue
    ress = np.array(ress)
    #dfile = open('dis_'+ressname.split('.pkl')[0]+'.txt','wb')
    #dfile.write("\n".join(discard))
    acs.pklwrite(ressname,ress) 

In [ ]:
elems = ['Al','Ca','C','Fe','K','Mg','Mn','Na','Ni','N','O','Si','S','Ti','V']
for elem in elems:
    w = wn.read(elem,dr = 12,apStarWavegrid=False)
    th = wn.tophat(elem,dr=12,apStarWavegrid=False)
    th = np.where(th == True)
    nw = pf.normweights(w)[th]
    resids = []
    for star in range(len(ress[0])):
        resids.append(pf.genresidual(nw,ress[:,star][th]))
    acs.pklwrite('{0}_cresiduals_order{1}.pkl'.format(elem,O),resids)
    plt.figure(figsize = (10,8))
    plt.hist(resids,bins = 50,range=(-0.1,0.1))
    plt.xlim(-0.1,0.1)
    plt.xlabel('Weighted fit residual of star for {0}'.format(elem))
    plt.ylabel('Count')
    plt.title('Residual from TEFF, LOGG, METALS fit for {0}'.format(elem))
    plt.savefig(dir+'{0}_cresiduals_order{1}.png'.format(elem,O))